# Day 14

Time to re-fuel at Saturn. We can use the raw materials from Saturn's rings, as long as we follow the formula's that are given to use

## Part 1

What's the minimum number of `ORE` needed to produce 1 `FUEL`?

In [1]:
import math
from typing import Dict, List, Union, Tuple

In [56]:
class Nanofactory:
    def __init__(self, formulas: str):
        self.formula_dict = self.make_formula_dict(formulas)
        self.materials_count_dict = self.make_materials_count_dict(self.formula_dict)
        self.materials_needed_dict = self.make_materials_needed_dict()
    
    @staticmethod
    def make_formula_dict(formulas: str) -> Dict[Tuple[int, str], List[Tuple[int, str]]]:
        components = [component.strip() for formula in formulas.split("\n") for component in formula.split("=>")]

        input_components = [component.split(", ") for component in components[0::2]]
        output_components = [tuple(component.split(" ")) for component in components[1::2]]
        # Convert output amounts to integers
        output_components = [(int(component[0]), component[1]) for component in output_components]

        formula_dict = dict(zip(output_components, input_components))

        for output_component in formula_dict:
            formula_dict[output_component] = [
                tuple(component.split(" ")) for component in  formula_dict[output_component]
            ]
            formula_dict[output_component] = [
                (int(component[0]), component[1]) for component in formula_dict[output_component]
            ]
        
        return formula_dict
    
    @staticmethod
    def make_materials_count_dict(formula_dict: Dict[tuple, List[tuple]]) -> Dict[str, int]:
        materials = [key[1] for key in formula_dict]
        material_amounts = [0 for _ in range(len(materials))]
        material_amount_dict = dict(zip(materials, material_amounts))
        return material_amount_dict
    
    def make_materials_needed_dict(self) -> Dict[str, int]:
        materials_needed_dict = self.materials_count_dict.copy()
        # Start out with one FUEL so we can work backwards
        materials_needed_dict["FUEL"] = 1
        # We'll need to know how much ORE we needed to get to 1 FUEL
        # Doesn't need to be in materials_count_dict since it's a limitless resource
        materials_needed_dict["ORE"] = 0
        return materials_needed_dict
    
    def find_needed_materials(self) -> List[Tuple[int, str]]:
        return [
            (count - self.materials_count_dict.get(material), material)
            for material, count in self.materials_needed_dict.items()
            if material != "ORE"
            and self.materials_count_dict.get(material) < count
        ]
    
    def translate_material_formula(self, material: Tuple[int, str], debug: bool = False):
        """Given a material, translate it to it's formula. Subtract the original formula
        from the materials_needed_dict after translation. Add the formula requirements
        to the materials_needed_dict. Add the total material after translation to the
        materials_count_dict
        """
        material_amount, material_name = material

        for formula in self.formula_dict:
            formula_amount, formula_name = formula
            if formula_name == material_name:
                formula_count = math.ceil(material_amount / formula_amount)
                if debug:
                    print(f"****Using {formula} formula {formula_count} time(s) for {material} material")
                    print(f"****Adding {formula_amount * formula_count} {material_name} to the count_dict")
                self.materials_count_dict[material_name] += formula_amount * formula_count
                for material in self.formula_dict.get(formula):
                    material_amount, material_name = material
                    self.materials_needed_dict[material_name] += material_amount * formula_count
                    if debug:
                        print(f"****Adding {material_amount * formula_count} {material_name} to the needed_dict")
                # Just want to do this for one material at a time
                break
    
    def get_ore_requirements(self, debug: bool = False) -> int:
        """Return the minimum number of ORE required to get 1 FUEL"""
        needed_materials = self.find_needed_materials()
        while needed_materials:
            if debug:
                print(f"Need materials to make {needed_materials}")
            for material in needed_materials:
                if debug:
                    print(f"**Get formula for {material}")
                    print(f"**Needed materials before: {self.materials_needed_dict}")
                    print(f"**Have materials before: {self.materials_count_dict}")
                self.translate_material_formula(material)
                if debug:
                    print(f"**Needed materials after: {self.materials_needed_dict}")
                    print(f"**Have materials after: {self.materials_count_dict}")
            needed_materials = self.find_needed_materials()
        
        return self.materials_needed_dict.get("ORE")

In [57]:
test_formulas = """10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL
"""
# Test the nitty gritty operations
test_nanofactory = Nanofactory(test_formulas)
assert test_nanofactory.materials_needed_dict == {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'FUEL': 1, 'ORE': 0}
assert test_nanofactory.materials_count_dict == {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'FUEL': 0}
assert test_nanofactory.find_needed_materials() == [(1, 'FUEL')]
test_nanofactory.translate_material_formula((1, 'FUEL'))
assert test_nanofactory.materials_needed_dict == {'A': 7, 'B': 0, 'C': 0, 'D': 0, 'E': 1, 'FUEL': 1, 'ORE': 0}
assert test_nanofactory.materials_count_dict == {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'FUEL': 1}
assert test_nanofactory.find_needed_materials() == [(7, 'A'), (1, 'E')]
test_nanofactory.translate_material_formula((7, 'A'))
test_nanofactory.translate_material_formula((1, 'E'))
assert test_nanofactory.materials_needed_dict == {'A': 14, 'B': 0, 'C': 0, 'D': 1, 'E': 1, 'FUEL': 1, 'ORE': 10}
assert test_nanofactory.materials_count_dict == {'A': 10, 'B': 0, 'C': 0, 'D': 0, 'E': 1, 'FUEL': 1}
assert test_nanofactory.find_needed_materials() == [(4, 'A'), (1, 'D')]
test_nanofactory.translate_material_formula((4, 'A'))
test_nanofactory.translate_material_formula((1, 'D'))
assert test_nanofactory.materials_needed_dict == {'A': 21, 'B': 0, 'C': 1, 'D': 1, 'E': 1, 'FUEL': 1, 'ORE': 20}
assert test_nanofactory.materials_count_dict == {'A': 20, 'B': 0, 'C': 0, 'D': 1, 'E': 1, 'FUEL': 1}
assert test_nanofactory.find_needed_materials() == [(1, 'A'), (1, 'C')]
test_nanofactory.translate_material_formula((1, 'A'))
test_nanofactory.translate_material_formula((1, 'C'))
assert test_nanofactory.materials_needed_dict == {'A': 28, 'B': 1, 'C': 1, 'D': 1, 'E': 1, 'FUEL': 1, 'ORE': 30}
assert test_nanofactory.materials_count_dict == {'A': 30, 'B': 0, 'C': 1, 'D': 1, 'E': 1, 'FUEL': 1}
assert test_nanofactory.find_needed_materials() == [(1, 'B')]
test_nanofactory.translate_material_formula((1, 'B'))
assert test_nanofactory.materials_needed_dict == {'A': 28, 'B': 1, 'C': 1, 'D': 1, 'E': 1, 'FUEL': 1, 'ORE': 31}
assert test_nanofactory.materials_count_dict == {'A': 30, 'B': 1, 'C': 1, 'D': 1, 'E': 1, 'FUEL': 1}
assert test_nanofactory.find_needed_materials() == []

In [58]:
# Test the big picture
test_formulas = """10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL"""
test_nanofactory = Nanofactory(test_formulas)
assert test_nanofactory.get_ore_requirements() == 31

In [59]:
test_formulas = """9 ORE => 2 A
8 ORE => 3 B
7 ORE => 5 C
3 A, 4 B => 1 AB
5 B, 7 C => 1 BC
4 C, 1 A => 1 CA
2 AB, 3 BC, 4 CA => 1 FUEL"""
test_nanofactory = Nanofactory(test_formulas)
assert test_nanofactory.get_ore_requirements() == 165

In [60]:
test_formulas = """157 ORE => 5 NZVS
165 ORE => 6 DCFZ
44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL
12 HKGWZ, 1 GPVTF, 8 PSHF => 9 QDVJ
179 ORE => 7 PSHF
177 ORE => 5 HKGWZ
7 DCFZ, 7 PSHF => 2 XJWVT
165 ORE => 2 GPVTF
3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF => 8 KHKGT"""
test_nanofactory = Nanofactory(test_formulas)
assert test_nanofactory.get_ore_requirements() == 13312

In [61]:
test_formulas = """2 VPVL, 7 FWMGM, 2 CXFTF, 11 MNCFX => 1 STKFG
17 NVRVD, 3 JNWZP => 8 VPVL
53 STKFG, 6 MNCFX, 46 VJHF, 81 HVMC, 68 CXFTF, 25 GNMV => 1 FUEL
22 VJHF, 37 MNCFX => 5 FWMGM
139 ORE => 4 NVRVD
144 ORE => 7 JNWZP
5 MNCFX, 7 RFSQX, 2 FWMGM, 2 VPVL, 19 CXFTF => 3 HVMC
5 VJHF, 7 MNCFX, 9 VPVL, 37 CXFTF => 6 GNMV
145 ORE => 6 MNCFX
1 NVRVD => 8 CXFTF
1 VJHF, 6 MNCFX => 4 RFSQX
176 ORE => 6 VJHF"""
test_nanofactory = Nanofactory(test_formulas)
assert test_nanofactory.get_ore_requirements() == 180697

In [62]:
test_formulas = """171 ORE => 8 CNZTR
7 ZLQW, 3 BMBT, 9 XCVML, 26 XMNCP, 1 WPTQ, 2 MZWV, 1 RJRHP => 4 PLWSL
114 ORE => 4 BHXH
14 VRPVC => 6 BMBT
6 BHXH, 18 KTJDG, 12 WPTQ, 7 PLWSL, 31 FHTLT, 37 ZDVW => 1 FUEL
6 WPTQ, 2 BMBT, 8 ZLQW, 18 KTJDG, 1 XMNCP, 6 MZWV, 1 RJRHP => 6 FHTLT
15 XDBXC, 2 LTCX, 1 VRPVC => 6 ZLQW
13 WPTQ, 10 LTCX, 3 RJRHP, 14 XMNCP, 2 MZWV, 1 ZLQW => 1 ZDVW
5 BMBT => 4 WPTQ
189 ORE => 9 KTJDG
1 MZWV, 17 XDBXC, 3 XCVML => 2 XMNCP
12 VRPVC, 27 CNZTR => 2 XDBXC
15 KTJDG, 12 BHXH => 5 XCVML
3 BHXH, 2 VRPVC => 7 MZWV
121 ORE => 7 VRPVC
7 XCVML => 6 RJRHP
5 BHXH, 4 VRPVC => 5 LTCX"""
test_nanofactory = Nanofactory(test_formulas)
test_nanofactory.get_ore_requirements()
assert test_nanofactory.get_ore_requirements() == 2210736

In [81]:
# Puzzle input
formulas = """3 PTZH, 14 MHDKS, 9 MPBVZ => 4 BDRP
4 VHPGT, 12 JSPDJ, 1 WNSC => 2 XCTCF
174 ORE => 4 JVNH
7 JVNH => 4 BTZH
12 XLNZ, 1 CZLDF => 8 NDHSR
1 VDVQ, 1 PTZH => 7 LXVZ
1 ZDQRT => 5 KJCJL
2 SGDXK, 6 VDVQ, 1 RLFHL => 7 GFNQ
8 JFBD => 5 VDVQ
1 SGDXK => 6 ZNBSR
2 PNZD, 1 JFBD => 7 TVRMW
11 TRXG, 4 CVHR, 1 VKXL, 63 GFNQ, 1 MGNW, 59 PFKHV, 22 KFPT, 3 KFCJC => 1 FUEL
6 BTZH => 8 GTWKH
5 WHVKJ, 1 QMZJX => 6 XLNZ
18 JSPDJ, 11 QMZJX => 5 RWQC
2 WFHXK => 4 JSPDJ
2 GHZW => 3 RLFHL
4 WHVKJ, 2 RWQC, 2 PTZH => 8 WNSC
1 QPJVR => 2 VFXSL
1 NCMQC => 6 GDLFK
199 ORE => 5 PNZD
2 RZND, 1 GTWKH, 2 VFXSL => 1 WHVKJ
1 VDVQ => 8 WFHXK
2 VFXSL => 4 VHMT
21 SBLQ, 4 XLNZ => 6 MGNW
6 SGDXK, 13 VDVQ => 9 NBSMG
1 SLKRN => 5 VKXL
3 ZNBSR, 1 WNSC => 1 TKWH
2 KJCJL => 6 LNRX
3 HPSK, 4 KZQC, 6 BPQBR, 2 MHDKS, 5 VKXL, 13 NDHSR => 9 TRXG
1 TKWH, 36 BDRP => 5 BNQFL
2 BJSWZ => 7 RZND
2 SLKRN, 1 NDHSR, 11 PTZH, 1 HPSK, 1 NCMQC, 1 BNQFL, 10 GFNQ => 2 KFCJC
3 LXVZ, 9 RWQC, 2 KJCJL => 7 VHPGT
2 GTWKH, 1 LNRX, 2 RZND => 1 MHDKS
18 RZND, 2 VHPGT, 7 JSPDJ => 9 NCMQC
2 NBSMG, 3 KJCJL => 9 BPQBR
124 ORE => 1 JFBD
1 QPJVR, 2 QMZJX => 4 SGDXK
4 BPQBR, 1 LNRX => 2 KZQC
1 KJCJL, 15 GTWKH => 2 SBLQ
1 ZDQRT, 3 CZLDF, 10 GDLFK, 1 BDRP, 10 VHMT, 6 XGVF, 1 RLFHL => 7 CVHR
1 KZQC => 8 MPBVZ
27 GRXH, 3 LNRX, 1 BPQBR => 6 XGVF
1 XCTCF => 6 KFPT
7 JFBD => 4 GHZW
19 VHPGT => 2 SLKRN
9 JFBD, 1 TVRMW, 10 BTZH => 6 BJSWZ
6 ZNBSR => 4 PTZH
1 JSPDJ, 2 BHNV, 1 RLFHL => 3 QMZJX
2 RCWX, 1 WNSC => 4 GRXH
2 TKWH, 5 NCMQC, 9 GRXH => 3 HPSK
32 KZQC => 5 RCWX
4 GHZW, 1 TVRMW => 1 QPJVR
2 QPJVR, 8 GHZW => 5 ZDQRT
1 VDVQ, 1 WFHXK => 6 BHNV
1 ZNBSR, 6 TKWH => 8 CZLDF
1 MGNW => 5 PFKHV
"""
nanofactory = Nanofactory(formulas)
nanofactory.get_ore_requirements()

301997

## Part 2

Great, now that we now how much ore it takes to produce one fuel, we check our cargo hold to see how much ORE we can hold -- 1 trillion ORE (Dr. Evil smile :smiling_imp:)

Once we've collected 1 trillion ORE, how much fuel can we make?

So now, ORE is not an unlimited resource -- we only have one trillion of it. Given that constraint, how much fuel can we produce until we run out of ORE?

In [79]:
test_formulas = """171 ORE => 8 CNZTR
7 ZLQW, 3 BMBT, 9 XCVML, 26 XMNCP, 1 WPTQ, 2 MZWV, 1 RJRHP => 4 PLWSL
114 ORE => 4 BHXH
14 VRPVC => 6 BMBT
6 BHXH, 18 KTJDG, 12 WPTQ, 7 PLWSL, 31 FHTLT, 37 ZDVW => 1 FUEL
6 WPTQ, 2 BMBT, 8 ZLQW, 18 KTJDG, 1 XMNCP, 6 MZWV, 1 RJRHP => 6 FHTLT
15 XDBXC, 2 LTCX, 1 VRPVC => 6 ZLQW
13 WPTQ, 10 LTCX, 3 RJRHP, 14 XMNCP, 2 MZWV, 1 ZLQW => 1 ZDVW
5 BMBT => 4 WPTQ
189 ORE => 9 KTJDG
1 MZWV, 17 XDBXC, 3 XCVML => 2 XMNCP
12 VRPVC, 27 CNZTR => 2 XDBXC
15 KTJDG, 12 BHXH => 5 XCVML
3 BHXH, 2 VRPVC => 7 MZWV
121 ORE => 7 VRPVC
7 XCVML => 6 RJRHP
5 BHXH, 4 VRPVC => 5 LTCX"""
test_nanofactory = Nanofactory(test_formulas)

In [80]:
# This is a very inefficient way to do it but it gives the right answer :shrug:
fuel_generated = 0
while test_nanofactory.get_ore_requirements() < 1000000000000:
    fuel_generated += 1
    ore_requirements = test_nanofactory.get_ore_requirements()
    # Increment another FUEL
    test_nanofactory.materials_needed_dict["FUEL"] += 1

assert fuel_generated == 460664

460664


In [82]:
# If this **never** runs, then we can try _incrementing_/_decrementing_ the FUEL needed
# until we hit an ORE requirement over/under 1000000000000
nanofactory = Nanofactory(formulas)
fuel_generated = 0
while nanofactory.get_ore_requirements() < 1000000000000:
    fuel_generated += 1
    ore_requirements = nanofactory.get_ore_requirements()
    # Increment another FUEL
    nanofactory.materials_needed_dict["FUEL"] += 1

fuel_generated

6216589